In [13]:
import pandas as pd
import plotly.graph_objects as go
import os
import cv2
from mediapipe.python.solutions import pose as mp_pose
import mediapipe as mp
import plotly.express as px
import matplotlib.pyplot as plt

_PRESENCE_THRESHOLD = 0.5
_VISIBILITY_THRESHOLD = 0.5


def plot_landmarks(
    landmark_list,
    connections=None,
):
    if not landmark_list:
        return
    plotted_landmarks = {}
    for idx, landmark in enumerate(landmark_list.landmark):
        if (
            landmark.HasField("visibility")
            and landmark.visibility < _VISIBILITY_THRESHOLD
        ) or (
            landmark.HasField("presence") and landmark.presence < _PRESENCE_THRESHOLD
        ):
            continue
        plotted_landmarks[idx] = (-landmark.z, landmark.x, -landmark.y)
    if connections:
        out_cn = []
        num_landmarks = len(landmark_list.landmark)
        # Draws the connections if the start and end landmarks are both visible.
        for connection in connections:
            start_idx = connection[0]
            end_idx = connection[1]
            if not (0 <= start_idx < num_landmarks and 0 <= end_idx < num_landmarks):
                raise ValueError(
                    f"Landmark index is out of range. Invalid connection "
                    f"from landmark #{start_idx} to landmark #{end_idx}."
                )
            if start_idx in plotted_landmarks and end_idx in plotted_landmarks:
                landmark_pair = [
                    plotted_landmarks[start_idx],
                    plotted_landmarks[end_idx],
                ]
                out_cn.append(
                    dict(
                        xs=[landmark_pair[0][0], landmark_pair[1][0]],
                        ys=[landmark_pair[0][1], landmark_pair[1][1]],
                        zs=[landmark_pair[0][2], landmark_pair[1][2]],
                    )
                )
        cn2 = {"xs": [], "ys": [], "zs": []}
        for pair in out_cn:
            for k in pair.keys():
                cn2[k].append(pair[k][0])
                cn2[k].append(pair[k][1])
                cn2[k].append(None)

    df = pd.DataFrame(plotted_landmarks).T.rename(columns={0: "z", 1: "x", 2: "y"})
    df["lm"] = df.index.map(lambda s: mp_pose.PoseLandmark(s).name).values
    fig = (
        px.scatter_3d(df, x="z", y="x", z="y", hover_name="lm")
        .update_traces(marker={"color": "red"})
        .update_layout(
            margin={"l": 0, "r": 0, "t": 0, "b": 0},
            scene={"camera": {"eye": {"x": 2.1, "y": 0, "z": 0}}},
        )
    )
    fig.add_traces(
        [
            go.Scatter3d(
                x=cn2["xs"],
                y=cn2["ys"],
                z=cn2["zs"],
                mode="lines",
                line={"color": "black", "width": 5},
                name="connections",
            )
        ]
    )

    return fig

In [14]:
path = 'C://python//train//fall/video1/fall-01-cam0-rgb-115.png'
# a = os.listdir(path)

In [15]:
#find filename in a


In [16]:
img = cv2.imread(path, cv2.IMREAD_COLOR)

In [17]:
pose = mp_pose.Pose()
mpDraw = mp.solutions.drawing_utils

In [18]:
pose.reset()

In [19]:
result = pose.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [20]:
def getPoint(name):
    if name.lower() == 'neck':
        return round(0.5* (getPoint('RIGHT_SHOULDER') + getPoint('LEFT_SHOULDER')), 10)
    return result.pose_world_landmarks.landmark[mp_pose.PoseLandmark._member_map_[name]].y

In [21]:
getPoint('neck')

0.0807434581

In [22]:
getPoint('RIGHT_SHOULDER')

-0.08275442570447922

In [23]:
getPoint('RIGHT_ANKLE')

-0.04665115848183632

In [24]:
getPoint('LEFT_ANKLE')

0.1479930281639099

In [25]:
getPoint('LEFT_SHOULDER')

0.24424134194850922

In [26]:

plot_landmarks(
        result.pose_world_landmarks,  mp_pose.POSE_CONNECTIONS)